This notebook is used to extract the wikipedia urls for the food ontology terms in the ClaimsKG dataset.

In [17]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/ClaimsKG.csv')
df2 = pd.read_csv('data/ClaimsKG2.csv')
frames = [df, df2]
df = pd.concat(frames)
df = df[['Claim', 'Food','Phenotype','Relationship-effect','Food Ontology Term']]
df = df.dropna(subset = ['Claim'])
df.index = range(0,len(df))
print(len(df))
df.head()

260


,Claim,Food,Phenotype,Relationship-effect,Food Ontology Term
0,ALA contributes to the maintenance of normal b...,Alpha-linolenic acid (ALA),Normal blood cholesterol,Maintenance of a function,https://foodb.ca/compounds/FDB012462
1,Activated charcoal contributes to reducing exe...,Activated charcoal,Excessive flatulence,Enhancing a function,https://foodb.ca/compounds/FDB008898
2,Barley grain fibre contributes to an increase ...,Barley grain fibre,Increase in faecal bulk,Enhancing a function,https://foodb.ca/foods/FOOD00088
3,Beta-glucans contribute to the maintenance of ...,Beta-glucans,Normal blood cholesterol,Maintenance of a function,https://foodb.ca/compounds/FDB005762
4,Betaine contributes to normal homocysteine met...,Betaine,Normal blood homocysteine,Maintenance of a function,https://foodb.ca/compounds/FDB009020


In [38]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
def extract_wikipedia_urls():
    wiki_ids = []
    seen_foodb = []
    for i in range(len(df)):
        wikipedia_id = []
        if df['Food Ontology Term'][i] is None:
            wiki_ids.append(wikipedia_id)
            continue
        try:
            urls = df['Food Ontology Term'][i].split('; ')
            for compound_url in urls:
                if compound_url in seen_foodb:
                    wikipedia_id.append('')
                    continue
                x_wikipedia_id = ''
                compound_url = str(df['Food Ontology Term'][i])
                options = Options()
                options.add_argument('--headless')
                options.add_argument('--disable-gpu')
                driver = webdriver.Chrome(chrome_options=options)
                driver.get(compound_url)
                page = driver.page_source
                driver.quit()
                soup = BeautifulSoup(page, 'html.parser')
                x_wikipedia_id = soup.findAll(text='Wikipedia ID')
                x_wikipedia_id = x_wikipedia_id[0].parent.parent.find('a')['href']
                seen_foodb.append(compound_url)

                wikipedia_id.append(x_wikipedia_id)
            wiki_ids.append(wikipedia_id)
        except:
            wiki_ids.append(wikipedia_id)
            continue
    df['foodDB_wiki'] = wiki_ids
    df['foodDB_wiki'] = df['foodDB_wiki'].replace(["['']"], None)
    df.sort_values('Food Ontology Term', inplace=True)
    df['foodDB_wiki'] = df.groupby('Food Ontology Term')['foodDB_wiki'].transform('first')
    df.sort_index(inplace=True)
    df['foodDB_wiki'] = df['foodDB_wiki'].replace('[]', None)
    df.to_csv('data/processed_claims.csv', index=False)

In [42]:
df = pd.read_csv('data/processed_claims.csv')
df.head()

,Claim,Food,Phenotype,Relationship-effect,Food Ontology Term,foodDB_wiki
0,ALA contributes to the maintenance of normal b...,Alpha-linolenic acid (ALA),Normal blood cholesterol,Maintenance of a function,https://foodb.ca/compounds/FDB012462,['http://en.wikipedia.org/wiki/Alpha-Linolenic...
1,Activated charcoal contributes to reducing exe...,Activated charcoal,Excessive flatulence,Enhancing a function,https://foodb.ca/compounds/FDB008898,['http://en.wikipedia.org/wiki/Activated_carbon']
2,Barley grain fibre contributes to an increase ...,Barley grain fibre,Increase in faecal bulk,Enhancing a function,https://foodb.ca/foods/FOOD00088,['http://en.wikipedia.org/wiki/Barley']
3,Beta-glucans contribute to the maintenance of ...,Beta-glucans,Normal blood cholesterol,Maintenance of a function,https://foodb.ca/compounds/FDB005762,['http://en.wikipedia.org/wiki/Glucan']
4,Betaine contributes to normal homocysteine met...,Betaine,Normal blood homocysteine,Maintenance of a function,https://foodb.ca/compounds/FDB009020,['http://en.wikipedia.org/wiki/Betaine']
